# Exercise 3: Durable Agent — Solution

Complete implementation combining OpenAI agents with Temporal for durability.

## Prerequisites

Ensure the Temporal dev server is running and `.env` contains your OpenAI credentials.

In [ ]:
%pip install --quiet temporalio openai rich

import json
import os
import asyncio
import uuid
from datetime import timedelta
from openai import OpenAI
from rich.console import Console
from temporalio import activity, workflow
from temporalio.client import Client
from temporalio.worker import Worker, UnsandboxedWorkflowRunner
from temporalio.common import RetryPolicy

console = Console()


def get_weather(location: str) -> str:
    """Return mock weather information for instructional tool calls."""
    weather_data = {
        "San Francisco": "sunny, 72°F",
        "New York": "cloudy, 65°F",
        "London": "rainy, 58°F",
        "Tokyo": "clear, 70°F",
    }
    weather = weather_data.get(location, "partly cloudy, 68°F")
    return f"The weather in {location} is {weather}"


def _safe_parse_arguments(arg_string: str) -> dict:
    try:
        return json.loads(arg_string)
    except json.JSONDecodeError:
        activity.logger.warning("Failed to parse tool arguments: %s", arg_string)
        return {}


@activity.defn
async def call_agent_with_tools(query: str, trace_id: str) -> str:
    """Invoke the OpenAI agent with tool calling wrapped as a Temporal activity."""
    activity.logger.info("🤖 Activity started")
    activity.logger.info("   Query: %s", query)
    activity.logger.info("   Trace ID: %s", trace_id)

    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_weather",
                "description": "Get the current weather for a location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "City name",
                        }
                    },
                    "required": ["location"],
                },
            },
        }
    ]

    messages = [
        {"role": "system", "content": "You are a helpful travel assistant."},
        {"role": "user", "content": query},
    ]

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools,
    )

    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls

    if tool_calls:
        activity.logger.info("🔧 Tool calls detected: %d", len(tool_calls))
        messages.append(response_message)
        for tool_call in tool_calls:
            payload = _safe_parse_arguments(tool_call.function.arguments)
            if tool_call.function.name == "get_weather" and payload.get("location"):
                tool_result = get_weather(**payload)
                activity.logger.info("   Calling: %s(%s)", tool_call.function.name, payload)
                messages.append(
                    {
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": tool_call.function.name,
                        "content": tool_result,
                    }
                )

        follow_up = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
        )
        final_message = follow_up.choices[0].message.content
    else:
        final_message = response_message.content

    activity.logger.info("✅ Activity completed")
    return final_message


@workflow.defn
class DurableAgentWorkflow:
    """Workflow coordinating the durable agent interaction."""

    @workflow.run
    async def run(self, query: str, trace_id: str) -> str:
        workflow.logger.info("🚀 Durable agent workflow started")
        workflow.logger.info("   Query: %s", query)
        workflow.logger.info("   Trace ID: %s", trace_id)

        result = await workflow.execute_activity(
            call_agent_with_tools,
            args=[query, trace_id],
            start_to_close_timeout=timedelta(seconds=30),
            retry_policy=RetryPolicy(
                initial_interval=timedelta(seconds=1),
                backoff_coefficient=2.0,
                maximum_interval=timedelta(seconds=10),
                maximum_attempts=3,
            ),
        )

        workflow.logger.info("✅ Workflow completed successfully")
        return result

## Run the Solution

Execute the workflow and observe durable behavior with retries.

In [ ]:
# Run the durable agent solution
async def run_solution() -> None:
    console.print("\n[bold cyan]🚀 Exercise 3: Durable Agent — Solution[/bold cyan]\n")

    trace_id = str(uuid.uuid4())
    console.print(f"[orange]Trace ID:[/orange] {trace_id}\n")

    client = await Client.connect("localhost:7233")
    task_queue = "durable-agent-queue"

    async with Worker(
        client,
        task_queue=task_queue,
        workflows=[DurableAgentWorkflow],
        activities=[call_agent_with_tools],
        workflow_runner=UnsandboxedWorkflowRunner(),
        debug_mode=True,
    ):
        query = "What's the weather like in San Francisco?"
        workflow_id = f"durable-agent-{trace_id}"

        console.print(f"[orange]Query:[/orange] {query}\n")

        result = await client.execute_workflow(
            DurableAgentWorkflow.run,
            args=[query, trace_id],
            id=workflow_id,
            task_queue=task_queue,
        )

    console.print(f"\n[bold green]🤖 Agent Response:[/bold green]\n{result}\n")
    console.print(
        f"[orange]View in Temporal UI:[/orange] "
        f"http://localhost:8233/namespaces/default/workflows/{workflow_id}"
    )
    console.print(f"[orange]Trace ID for correlation:[/orange] {trace_id}\n")

try:
    loop = asyncio.get_running_loop()
except RuntimeError:
    asyncio.run(run_solution())
else:
    import nest_asyncio
    nest_asyncio.apply()
    task = loop.create_task(run_solution())
    await task

## Source Code

The workflow and activity implementations are defined above in the first code cell.